# PyCaret (usando datos por Hora)

In [2]:
# Agregamos Prophet a PyCaret, hay que reiniciar el Kernell despues
!pip install prophet
!pip install --pre pycaret
!pip install pingouin
!pip install openpyxl


import numpy as np
import pandas as pd
import warnings
from pycaret.time_series import TSForecastingExperiment
from matplotlib import pyplot as plt

# Configuracion para graficas
fig_kwargs = {
#     "renderer": "notebook",
    "renderer": "png",
    "width": 800,
    "height": 500,
}

# warnings.filterwarnings('ignore', category=ValueWarning)
warnings.filterwarnings('ignore')

# Vamos a suprimir la notacion cientifica
pd.set_option("display.float_format", lambda x:"%.2f" %x)

# Usa una fuente que tengas instalada
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'DejaVu Sans'  


## Carga de los Datos por Dia

In [3]:
# Cargamos Demanda por Dia
data = pd.read_excel('Data/data_hora.xlsx')
data['Hora'] = pd.to_datetime(data['Hora'])
data.set_index('Hora', inplace=True)

print(data.info())

* A diferencia de como hicimos en el TP#1, aqui cargamos los datos originales por hora en lugar de por dia
* Serian unos 26000 registros en lugar de 1100

# EDA Experiment

In [4]:
# Create an EDA experiment ----
eda = TSForecastingExperiment()

# Con indice implicito. Tambien probar con indice explicito luego!!!!
eda.setup(
    data=data,
    target='Demanda',
#     index=data.index,
    fh=31*24,
    # Set defaults for the plots ----
    fig_kwargs=fig_kwargs,
    session_id=1515,
)

# Vemos 
eda.plot_model(plot="ccf", fig_kwargs=fig_kwargs)

* No se observa ninguna correlacion entre los lags de las distintas Series, decidimos incluir las tres en el modelo

# AutoML

In [5]:
# Instanciamos el Experimento y configuramos la Pipeline
exp = TSForecastingExperiment()

# enforce_exogenous=False --> Use multivariate forecasting when model supports it, else use univariate forecasting
exp.setup(
    data=data, target='Demanda', fh=31, enforce_exogenous=False,
    fig_kwargs=fig_kwargs, session_id=1515
)

* Instanciamos el experimento, reservando para test los ultimos 31 registros (Diciembre 2022)
* Vemos como en la linea #24 ya se detecta la Estacionalidad Diara (ya que los datos son diarios)

In [ ]:
# Seteo con cuantos modelos me quiero quedar
n = 3
best_baseline_models = exp.compare_models(n_select=n, turbo=False,exclude=['tbats', 'bats', 'arima'])

# Guardamos la grilla de metricas
compare_metrics = exp.pull()

* Vemos que el modelo Prohet arroja mejores resultados en todas las metricas.
* Igualmente nos quedamos con los 3 mejores modelos para Ensamblarlos luego (Hubert, KNN, Random Forest)
* Recordemos que el objetivo de esta notebook es ver lo sencillo que es aplicar esta libreia de AutoML llamada PyCaret, y no encontrar el modelo mas optimo

In [ ]:
# # Tuneamos los Hyperparametros de los mejores modelos

# Si solo nos quedamos con el mejor
if n==1:
    best_tuned_models = exp.tune_model(best_baseline_models)
# Si nos quedamos con mas de uno, para hacer un blend
else:
    best_tuned_models = [exp.tune_model(model) for model in best_baseline_models]

* Tambien tiene una opcion para Auto Tunear los valores de los Hyperparametros, parecido a lo que vimos como GridSearch

In [ ]:
# Aplicamos diferente peso a cada modelo
top_model_metrics = compare_metrics.iloc[0:n]['MAE']
display(top_model_metrics)

# Si es un solo modelo, el peso es 1
if n==1:
    top_model_weights = pd.Series(1, name='MAE')
# Si son varios modelos, se calcula el peso de cada uno
else:
    top_model_weights = 1 - top_model_metrics/top_model_metrics.sum()
display(top_model_weights)

* Ponderamos a cada modelo segun su efectivdad, y elegimos el Mean Absolute Error como Metrica

In [ ]:
# Combinamos los modelos con sus respectivos pesos 
if n==1:
    blender = best_tuned_models
else:
    blender = exp.blend_models(best_tuned_models, method='mean', weights=top_model_weights.values.tolist())

* Vemos que tambien es sencillo hacer el Blend, y agregarle distintos pesos a cada modelo

In [ ]:
# Predicted vs Reality
exp.plot_model(estimator=blender, fig_kwargs=fig_kwargs)

* Analizamos graficamente la efectividad de la prediccion en los datos de Test (Dicimebre 2022)
* Esto lo seteamos en el parametro fh cuando creamos el Setup del modelo en las primeras lineas

In [ ]:
# NO FUNCIONA
# Finalizamos el modelo, entrenando con todos los datos
final_model = exp.finalize_model(blender)
# print(exp.predict_model(final_model))

# Forexast para los datos del futuro
# exp.plot_model(final_model)


* Finalizamos el modelo entrenandolo con todos los datos disponibles (Train + Test)
* Predecimos los valores futuros todavia desconocidos (Enero 2023) y los visualizamos
* Guardamos el modelo por si lo queremos usar en el futuro, sin tener que volver a crearlo desde cero

# Analisis de Residuos

In [ ]:
# Libreria importada del TP#1 para el analisis de residuos

# Instalacion y carga
# !pip install pingouin
import pingouin as pg
import statsmodels.api as sm
from scipy.stats import jarque_bera
from scipy.stats import shapiro
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import het_breuschpagan
from pmdarima.arima.utils import ndiffs

# Diferentes Tests de Normalidad
def residuos_normality(residuos):
  # Test Grafico
  pg.qqplot(residuos, dist='norm')
  plt.title('Normalidad de los Residuos')
  plt.show()

  # Test Analitico
  jarquebera = jarque_bera(residuos)[1]
  shapir = shapiro(residuos)[1]
  print('Shapiro p-value:', np.round(shapir, 3))
  print('Jarque-Bera p-value:', np.round(jarquebera, 3), '\n')

# Diferentes Tests de Autocorrelacion
def residuos_autocorrelation(residuos):
  # Test Grafico
  plot_acf(residuos, lags=len(residuos)-1)
  plt.title('Autocorrelacion de los Residuos')
  plt.show()
  # Tests Analiticos
  dw = sm.stats.stattools.durbin_watson(residuos)
  lb = sm.stats.acorr_ljungbox(residuos, lags=len(residuos)-1, return_df=True)
  print('Durbin-Watson (~2 = No-Autocorrelation):', np.round(dw, 3))
#   print('Ljung-Box p-value:', lb.lb_pvalue.values[3])

# Test Grafico de Homocedasticidad, el Analitco no lo pude hacer funcionar
def residuos_homocedasticity(residuos):
  # Grafico
  plt.scatter(range(len(residuos)), residuos)
  plt.axhline(y=0, color='r', linestyle='--')
  plt.xlabel('Tiempo')
  plt.ylabel('Residuos')
  plt.title('Homocedasticidad - Residuos a lo Largo del Tiempo')
  plt.show()

  # Analitico
  X = np.arange(len(residuos))
  X_with_const = sm.add_constant(X)
  print('Breusch-Pagan p-value (H0: Homocedasticity):', het_breuschpagan(residuos, X_with_const)[1].round(3))

# Llama a todos los test anteriores, mas la media
def residuos_evaluation(residuos):
  print('Media de los Residuos', np.round(residuos.mean(), 3))
  residuos_normality(residuos)
  residuos_autocorrelation(residuos)
  residuos_homocedasticity(residuos)
    
    
# Para obtener el d optimo a diferenciar
def diferenciacion(y):
  # Estimado de número de diferencias con ADF test:Dickey-Fuller
  n_adf = ndiffs(y, test='adf')  # -> 0

  # KPSS test (auto_arima default): Kwiatkowski-Phillips-Schmidt-Shin
  n_kpss = ndiffs(y, test='kpss')  # -> 0

  # PP test: Phillips-Perron
  n_pp = ndiffs(y, test='pp')  # -> 0

  print('Estimado de número de diferencias con KPSS test')
  print(n_kpss)
  print('Estimado de número de diferencias con ADF test')
  print(n_adf)
  print('Estimado de número de diferencias con PP test')
  print(n_pp)

In [ ]:
# PyCaret no trae los residuos del modelo, los calculo a mano
residuos_blender = exp.predict_model(blender)['y_pred'] - exp.get_config('y_test')

# # Llamo a la funcion creada del TP#1
residuos_evaluation((residuos_blender))

# Nombres de todos los modelos
# best_baseline_models, best_tuned_models, blender, final_model

* Utilizamos una Funcion creada en el TP#1 extendida para analizar los Residuos del Modelo Auto-Blend creado, y vemos:
* Tanto los tests de Normalidad de los Residuos de Shapuro y de Jarque-Bera son medio border, la Normalidad de los mismos es cuestionable
* El test de Homocedasticidad de Breusche-Pagan nos indica que no hay suficiente evidencia para rechazar la H0 de que los residuos son Homocedasticos. Sin embargo esto no se puede apreciar bien en el Test Grafico
* El problema lo tenemos en la Autocorrelacion, la misma es indicada tanto en el test analitico de Durbin-Watson como en la grafica de PCAF (alta correlacion con el lag1). Recordemos que estas series **no estan diferenciadas**
* Por todo lo anterior podemos decir que la combinacion de modelos tuneados no pareciera ser optima
* Probamos el mismo Pipeline pero con los datos diferenciados, y tanto las metricas de performance como el analisis de diagnostico arrojaron mejores resultados. Compobamos que la diferenciacion es recomendad segun el test de KPSS

In [ ]:
# Chequeo sobre Diferenciacion en la Serie Original de Demanda 

print('Para Demanda:')
print(diferenciacion(data['Demanda']), '\n')
print('Para Viento:')
print(diferenciacion(data['Viento']))

* Confirmamos con un test de KPSS que conviene diferenciar ambas series (Demanda y Viento) al menos una vez
* Nos resulta raro que esto no haya sido captado automaticamente por PyCaret